# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.112961e+02     1.691104e+00
 * time: 0.06991696357727051
     1     1.128395e+01     9.977066e-01
 * time: 1.2015178203582764
     2    -1.153158e+01     1.123952e+00
 * time: 1.2973577976226807
     3    -3.408486e+01     7.835358e-01
 * time: 1.4231128692626953
     4    -4.783036e+01     5.540667e-01
 * time: 1.5987508296966553
     5    -5.705243e+01     2.026521e-01
 * time: 1.7099947929382324
     6    -5.983619e+01     1.534615e-01
 * time: 1.7965068817138672
     7    -6.096012e+01     6.100847e-02
 * time: 1.8827099800109863
     8    -6.144123e+01     4.354825e-02
 * time: 1.9958429336547852
     9    -6.172120e+01     5.144921e-02
 * time: 2.082169771194458
    10    -6.191512e+01     2.700266e-02
 * time: 2.1696338653564453
    11    -6.204655e+01     1.877064e-02
 * time: 2.255772829055786
    12    -6.211287e+01     1.332223e-02
 * time: 2.3412718772888184
    13    -6.214761e+01     1.301671e-02
 * time: 2.44013285636

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671067
    AtomicLocal         -18.8557659
    AtomicNonlocal      14.8522629
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485368 
    Xc                  -19.3336819

    total               -62.261666460349
